In [1]:
from arcgis.gis import GIS
from arcgis.features import Feature, FeatureSet
from arcgis.geometry import Envelope, Point
from datetime import datetime
from data_engineering.utils import (
    fetch_charging_stations, 
    fetch_hotcold_features, 
    fetch_hottest_features_by_extent, 
    fetch_traffic_accidents, 
    get_hotcold_layer, 
    get_live_traffic_item
)
import os
import pandas as pd
from urban_traffic.utils import (
    create_map, 
    create_traffic_map,
    evaluate_model,
    fetch_traffic_data, 
    filter_commute_cars, 
    generate_car_renderer, 
    generate_routes_renderer, 
    prepare_traffic,
    prepare_traffic_accidents,
    read_traffic_accidents_features_by_extent,
    read_traffic_features
)

### Additional dependencies
We load the necessary variables using `dotenv`.  
Please, ensure that you setup before: `pip install python-dotenv`  
Furthermore, put a `.env` file into the root of this repository.

In [2]:
from dotenv import load_dotenv

def load_vars():
    load_dotenv(dotenv_path="../../../../.env", override=True)
    return os.getenv("ARCGIS_API_KEY"), os.getenv("TRAFFIC_ACCIDENTS_FEATURES"), os.getenv("TRAFFIC_DATA_FILE"), os.getenv("TRAFFIC_FEATURES"), os.getenv("NETWORK_DATASET"), os.getenv("ELECTRIC_CAR_FEATURES")

In [3]:
api_key, traffic_accidents_features_filepath, traffic_data_filepath, traffic_features_filepath, network_dataset_filepath, electric_car_features_filepath = load_vars()

### Connect to your GIS portal

In [4]:
gis = GIS(api_key=api_key)

### Use a map for the area of interest

In [5]:
traffic_map_view = create_traffic_map(gis)
traffic_map_view

Map(center=[6465801.0751674855, 962955.6795531422], extent={'xmin': 962882.1493646716, 'ymin': 6465580.1136933…

In [6]:
map_view = create_map(gis, location="Frankenallee 355, Frankfurt am Main, Germany")
map_view

Map(center=[6463970.497258207, 959241.6442823887], extent={'xmin': 959131.7990409209, 'ymin': 6463772.48065509…

### Spatial filter using the area of interest

In [7]:
distance_in_meters = 150
lon, lat = map_view.center[:2]
traffic_features = read_traffic_features(traffic_features_filepath, lon=lon, lat=lat, meters=distance_in_meters)
traffic_features[["trip", "person", "vehicle_type"]]

,trip,person,vehicle_type
0,11661,7271,Bike
1,11661,7271,Bike
2,11661,7271,Bike
3,11661,7271,Bike
4,11661,7271,Bike
...,...,...,...
7969,1373,858,Bike
7970,1373,858,Bike
7971,1373,858,Bike
7972,1373,858,Bike


## Ingestion & Data Engineering

In [9]:
traffic_features.dtypes

OBJECTID           int64
SHAPE           geometry
trip               int64
person             int64
vehicle_type      object
trip_time         object
dtype: object

In [9]:
prepared_traffic_features = prepare_traffic(traffic_features.copy())
prepared_traffic_features

,OBJECTID,SHAPE,trip,person,hour,minute,second,bike,car,pedestrian
0,75210,"{""x"": 8.615455619000045, ""y"": 50.1027864670000...",11661,7271,1,52,28,1,0,0
1,75233,"{""x"": 8.615466635000075, ""y"": 50.1027515590000...",11661,7271,1,52,29,1,0,0
2,75258,"{""x"": 8.61547765000006, ""y"": 50.10271665200002...",11661,7271,1,52,30,1,0,0
3,75282,"{""x"": 8.615488666000033, ""y"": 50.1026817450000...",11661,7271,1,52,31,1,0,0
4,75306,"{""x"": 8.615499682000063, ""y"": 50.1026468380000...",11661,7271,1,52,32,1,0,0
...,...,...,...,...,...,...,...,...,...,...
7969,17228677,"{""x"": 8.618661074000045, ""y"": 50.1012395350000...",1373,858,23,32,58,1,0,0
7970,17228705,"{""x"": 8.618668207000042, ""y"": 50.1012061070000...",1373,858,23,32,59,1,0,0
7971,17228731,"{""x"": 8.618675339000049, ""y"": 50.1011726770000...",1373,858,23,33,0,1,0,0
7972,17228758,"{""x"": 8.618682471000056, ""y"": 50.1011392490000...",1373,858,23,33,1,1,0,0


In [10]:
prepared_traffic_features.dtypes

OBJECTID         int64
SHAPE         geometry
trip             int64
person           int64
hour             int32
minute           int32
second           int32
bike             int32
car              int32
pedestrian       int32
dtype: object

### Visualize commute traffic

In [11]:
commute_cars = filter_commute_cars(prepared_traffic_features.copy())
commute_cars

,OBJECTID,SHAPE,trip,person,hour,minute,second,bike,car,pedestrian
1294,2423183,"{""x"": 8.618313765000039, ""y"": 50.1029611390000...",4465,2771,8,4,8,0,1,0
1295,2423570,"{""x"": 8.618321629000036, ""y"": 50.1028414760000...",4465,2771,8,4,9,0,1,0
1296,2424352,"{""x"": 8.618350565000071, ""y"": 50.1027179640000...",4465,2771,8,4,10,0,1,0
1297,2424795,"{""x"": 8.618323860000032, ""y"": 50.1029196350000...",2228,1378,8,4,11,0,1,0
1298,2424953,"{""x"": 8.61837866600007, ""y"": 50.10259437100006...",4465,2771,8,4,11,0,1,0
...,...,...,...,...,...,...,...,...,...,...
3258,6734311,"{""x"": 8.615983551000056, ""y"": 50.1012082780000...",460,292,9,50,35,0,1,0
3259,6735153,"{""x"": 8.616021930000045, ""y"": 50.1010858360000...",460,292,9,50,36,0,1,0
3260,6735842,"{""x"": 8.61606094900003, ""y"": 50.10096347300003...",460,292,9,50,37,0,1,0
3261,6736537,"{""x"": 8.61609997000005, ""y"": 50.10084111000003...",460,292,9,50,38,0,1,0


In [12]:
commute_map_view = create_map(gis)
commute_cars.spatial.plot(commute_map_view, renderer=generate_car_renderer())
commute_map_view.zoom_to_layer(commute_cars)
commute_map_view

Map(center=[6465801.0751674855, 962955.6795531422], extent={'xmin': 959103.9242357736, 'ymin': 6463736.8591492…

## Visualize the traffic hot and cold spots
Using a traffic feature set and a pre-defined drawing info.

In [13]:
hotcold_spots_fset, drawing_info = fetch_hotcold_features(gis, commute_cars)
commute_map_view.content.add(hotcold_spots_fset, drawing_info=drawing_info, options={"opacity": 0.7})

## Advanced Spatial Analysis finding the closest charging station
We want to find one or more nearby charging stations from car positions based on travel time or travel distance. Once we've found the closest charging stations, we can display the best route to them and include the travel time, travel distance, and driving directions to each charging station in multiple languages.

We can use closest facility routing to build applications that:

- Find the closest hospital to an accident.
- Dispatch the two closest police cars to a crime scene.
- Find the three closest fire stations that can respond to a fire incident within five minutes' drive time.

In [33]:
import arcpy
from arcpy.management import CopyFeatures
from arcpy.na import AddLocations, GetNAClassNames, Solve

In [34]:
arcpy.env.overwriteOutput = True

In [35]:
def create_analysis_layer(charging_stations: pd.DataFrame, car_positions: pd.DataFrame):
    # Create the analysis layer using a local routing network dataset
    analysis_layer_name = "ClosestChargingStation"
    layer_result = arcpy.na.MakeClosestFacilityAnalysisLayer(network_dataset_filepath, analysis_layer_name)

    # Get the analysis class names ("Facilities", "Incidents", ...)
    analysis_layer = layer_result.getOutput(0)
    analysis_class_names = GetNAClassNames(analysis_layer)

    # Charging stations as facilities
    input_facilities = "memory/facilities"
    charging_stations.spatial.to_featureclass(input_facilities)
    AddLocations(analysis_layer, analysis_class_names["Facilities"], input_facilities)

    # Car positions as incidents
    input_incidents = "memory/incidents"
    car_positions.spatial.to_featureclass(input_incidents)
    AddLocations(analysis_layer, analysis_class_names["Incidents"], input_incidents)

    return analysis_layer

def solve_routing_problem(analysis_layer, nogo_areas: pd.DataFrame=None) -> pd.DataFrame:
    # Get the analysis class names ("Facilities", "Incidents", ...)
    analysis_class_names = GetNAClassNames(analysis_layer)

    if not None is nogo_areas:
        # Add nogo areas as polygon barriers
        input_barriers = "memory/barriers"
        nogo_areas.spatial.to_featureclass(input_barriers)
        AddLocations(analysis_layer, analysis_class_names["PolygonBarriers"], input_barriers)
    
    # Solve the routing problem
    solve_result = Solve(analysis_layer)

    # Extract the solved routes
    output_routes = "memory/routes"
    CopyFeatures(analysis_class_names["CFRoutes"], output_routes)
    return pd.DataFrame.spatial.from_featureclass(output_routes)

In [20]:
charging_stations = fetch_charging_stations(gis, extent=Envelope(commute_map_view.extent))
charging_stations

,OBJECTID,Ladeeinrichtungs_ID,Betreiber,Anzeigename__Karte_,Status,Art_der_Ladeeinrichtung,Anzahl_Ladepunkte,Nennleistung_Ladeeinrichtung__k,Inbetriebnahmedatum,Straße,...,Public_Key4,Steckertypen5,Nennleistung_Stecker5,EVSE_ID5,Public_Key5,Steckertypen6,Nennleistung_Stecker6,EVSE_ID6,Public_Key6,SHAPE
0,42679,1111949.0,On Charge GmbH,<NA>,In Betrieb,Normalladeeinrichtung,2.0,27.0,2023-08-29,Frankenallee,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"{""x"": 959243.1691000015, ""y"": 6463961.64850000..."
1,43006,1068595.0,Volkswagen Automobile Frankfurt GmbH,Automobile Frankfurt GmbH,In Betrieb,Schnellladeeinrichtung,1.0,75.0,2020-07-31,Mainzer Landstr.,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"{""x"": 960018.9545999989, ""y"": 6463911.66589999..."


In [21]:
car_positions = pd.DataFrame.spatial.from_featureclass(electric_car_features_filepath)
car_positions

,OBJECTID,Direction,SHAPE
0,27585,West,"{""x"": 944333.5036999993, ""y"": 6459804.1998, ""s..."
1,27586,North,"{""x"": 951553.8691999987, ""y"": 6495628.34189999..."
2,27587,East,"{""x"": 1012622.3944999985, ""y"": 6479589.8342999..."
3,27588,South,"{""x"": 957438.7252000012, ""y"": 6443890.12070000..."


In [39]:
analysis_layer = create_analysis_layer(charging_stations, car_positions)
analysis_layer

In [40]:
charging_routes = solve_routing_problem(analysis_layer)
charging_routes

,OBJECTID,FacilityID,FacilityRank,Name,IncidentCurbApproach,FacilityCurbApproach,IncidentID,StartTime,EndTime,StartTimeUTC,EndTimeUTC,Total_Minutes,Total_TravelTime,Total_Miles,Total_Kilometers,Total_TimeAt1KPH,Total_WalkTime,Total_TruckMinutes,Total_TruckTravelTime,SHAPE
0,1,1,1,Location 1 - Location 1,1,2,1,NaT,NaT,NaT,NaT,<NA>,10.859355,<NA>,15.348667,<NA>,<NA>,<NA>,<NA>,"{""hasM"": true, ""paths"": [[[944256.6121000014, ..."
1,2,1,1,Location 2 - Location 1,1,2,2,NaT,NaT,NaT,NaT,<NA>,22.814049,<NA>,30.006559,<NA>,<NA>,<NA>,<NA>,"{""hasM"": true, ""paths"": [[[951555.1268999986, ..."
2,3,1,1,Location 3 - Location 1,2,2,3,NaT,NaT,NaT,NaT,<NA>,36.108108,<NA>,55.501626,<NA>,<NA>,<NA>,<NA>,"{""hasM"": true, ""paths"": [[[1012640.0172000006,..."
3,4,1,1,Location 4 - Location 1,1,2,4,NaT,NaT,NaT,NaT,<NA>,12.35146,<NA>,17.923503,<NA>,<NA>,<NA>,<NA>,"{""hasM"": true, ""paths"": [[[957434.3684, 644388..."


In [41]:
charging_map = create_map(gis)
charging_stations.spatial.plot(charging_map, renderer=generate_car_renderer())
car_positions.spatial.plot(charging_map, renderer=generate_car_renderer())
charging_routes.spatial.plot(charging_map, renderer=generate_routes_renderer())
charging_map.zoom = 12
charging_map

Map(center=[6465801.0751674855, 962955.6795531422], extent={'xmin': 962882.1493646716, 'ymin': 6465580.1136933…

### Avoid the commute traffic areas

In [42]:
hottest_spots_fset, drawing_info = fetch_hottest_features_by_extent(gis, extent=Envelope(charging_map.extent))
charging_map.content.add(hottest_spots_fset, drawing_info=drawing_info, options={"opacity": 0.7})

In [43]:
# Solve it again
charging_routes = solve_routing_problem(analysis_layer, nogo_areas=hottest_spots_fset.sdf)

# Put it on a map
charging_map = create_map(gis)
charging_stations.spatial.plot(charging_map, renderer=generate_car_renderer())
charging_routes.spatial.plot(charging_map, renderer=generate_routes_renderer())
charging_map.content.add(hottest_spots_fset, drawing_info=drawing_info, options={"opacity": 0.7})
charging_map.zoom = 12
charging_map

Map(center=[6465801.0751674855, 962955.6795531422], extent={'xmin': 962882.1493646716, 'ymin': 6465580.1136933…

## Machine Learning: Evaluate the traffic accidents

In [44]:
#traffic_accidents = fetch_traffic_accidents(gis, extent=Envelope(traffic_map_view.extent))
traffic_accidents = read_traffic_accidents_features_by_extent(traffic_accidents_features_filepath, extent=Envelope(traffic_map_view.extent))
traffic_accidents

,uwochentag,ustunde,ukategorie,uart,utyp1,ulichtverh,ist_strasse,SHAPE
0,Montag,21,Unfall mit Leichtverletzten,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Einbiegen / Kreuzen-Unfall,Dunkelheit,trocken,"{""x"": 984303.808600001, ""y"": 6467846.5352, ""sp..."
1,Sonntag,11,Unfall mit Leichtverletzten,Zusammenstoß mit vorausfahrendem/wartendem Fah...,Unfall im Längsverkehr,Tageslicht,trocken,"{""x"": 972037.4670999981, ""y"": 6466849.5427, ""s..."
2,Samstag,18,Unfall mit Leichtverletzten,Unfall anderer Art,Abbiegeunfall,Dunkelheit,trocken,"{""x"": 964749.2127999999, ""y"": 6471931.29479999..."
3,Freitag,10,Unfall mit Leichtverletzten,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Abbiegeunfall,Tageslicht,trocken,"{""x"": 958189.1339000016, ""y"": 6468441.68129999..."
4,Freitag,15,Unfall mit Leichtverletzten,Zusammenstoß mit vorausfahrendem/wartendem Fah...,Unfall im Längsverkehr,Tageslicht,trocken,"{""x"": 968191.1779000014, ""y"": 6470081.7896, ""s..."
...,...,...,...,...,...,...,...,...
3767,Sonntag,20,Unfall mit Schwerverletzten,Zusammenstoß zwischen Fahrzeug und Fußgänger,sonstiger Unfall,Dunkelheit,trocken,"{""x"": 964584.5866999999, ""y"": 6477363.48409999..."
3768,Dienstag,18,Unfall mit Leichtverletzten,Unfall anderer Art,Unfall im Längsverkehr,Dunkelheit,trocken,"{""x"": 965963.7895999998, ""y"": 6474798.78069999..."
3769,Mittwoch,07,Unfall mit Schwerverletzten,Zusammenstoß mit mit entgegenkommendem Fahrzeug,Unfall im Längsverkehr,Tageslicht,nass/feucht/schlüpfrig,"{""x"": 967777.6383999996, ""y"": 6465421.83269999..."
3770,Freitag,14,Unfall mit Leichtverletzten,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Einbiegen / Kreuzen-Unfall,Tageslicht,trocken,"{""x"": 968845.1975000016, ""y"": 6471671.89940000..."


In [45]:
traffic_accidents["utyp1"].value_counts()

utyp1
Unfall im Längsverkehr           1286
Einbiegen / Kreuzen-Unfall        818
Abbiegeunfall                     481
sonstiger Unfall                  384
Fahrunfall                        328
Überschreitenunfall               281
Unfall durch ruhenden Verkehr     194
Name: count, dtype: Int64

### Validating the results
The accuracy tells us in how many cases the model correctly predicts the accident type.  
`60%` is reasonable for a multi-class problem with `7` classes.

In [46]:
# Categorical and numeric columns
categorical_cols = ["uwochentag", "uart", "ukategorie", "ulichtverh", "ist_strasse"]
numeric_cols = ["ustunde"]
target_col = "utyp1"
evaluate_model(traffic_accidents, categorical_cols, numeric_cols, target_col)

Accuracy: 0.64
Classification Report:
                                precision    recall  f1-score   support

                Abbiegeunfall       0.30      0.15      0.20       162
   Einbiegen / Kreuzen-Unfall       0.75      0.85      0.79       231
                   Fahrunfall       0.58      0.68      0.62        95
Unfall durch ruhenden Verkehr       0.66      0.57      0.61        67
       Unfall im Längsverkehr       0.78      0.86      0.82       388
             sonstiger Unfall       0.24      0.21      0.22       104
          Überschreitenunfall       0.53      0.61      0.57        85

                     accuracy                           0.64      1132
                    macro avg       0.55      0.56      0.55      1132
                 weighted avg       0.61      0.64      0.62      1132



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['uwochentag', 'uart',
                                                   'ukategorie', 'ulichtverh',
                                                   'ist_strasse']),
                                                 ('num', StandardScaler(),
                                                  ['ustunde'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [48]:
prepared_traffic_accidents, target_col = prepare_traffic_accidents(traffic_accidents)
prepared_traffic_accidents[target_col].value_counts()

utyp1_grouped
Längsverkehr         1808
Kreuzung/Abbiegen    1299
Sonstige              665
Name: count, dtype: int64

In [49]:
evaluate_model(prepared_traffic_accidents, categorical_cols, numeric_cols, target_col)

Accuracy: 0.78
Classification Report:
                    precision    recall  f1-score   support

Kreuzung/Abbiegen       0.85      0.73      0.78       393
     Längsverkehr       0.79      0.87      0.83       550
         Sonstige       0.60      0.60      0.60       189

         accuracy                           0.78      1132
        macro avg       0.75      0.73      0.74      1132
     weighted avg       0.78      0.78      0.78      1132



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['uwochentag', 'uart',
                                                   'ukategorie', 'ulichtverh',
                                                   'ist_strasse']),
                                                 ('num', StandardScaler(),
                                                  ['ustunde'])])),
                ('classifier', RandomForestClassifier(random_state=42))])